# 0. Magic

In [35]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# 1. Import

In [36]:
from pathlib import Path
from IPython.core.debugger import set_trace
from fastai import datasets
import pickle, gzip, math, torch, matplotlib as mpl
import matplotlib.pyplot as plt
from torch import tensor
import operator

In [37]:
# tol = tolerance
def test_near_zero(a, tol=1e-3): assert a.abs() < tol, f"Near Zero: {a}"

# 2. Data

In [38]:
MNIST_URL='http://deeplearning.net/data/mnist/mnist.pkl'

In [39]:
def get_data():
    path = datasets.download_data(MNIST_URL, ext='.gz')
    with gzip.open(path, 'rb') as f:
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')
    return map(tensor, (x_train, y_train, x_valid, y_valid))

In [40]:
x_train, y_train, x_valid, y_valid = get_data()

# 3. เตรียมข้อมูล

หา mean และ standard deviation ของข้อมูล Training Set

In [41]:
train_mean, train_std = x_train.mean(), x_train.std()
train_mean, train_std

(tensor(0.1304), tensor(0.3073))

Normalize คือการทำให้ข้อมูล มี mean = 0 และ standard deviation (std) = 1 ด้วยกัน ลบด้วย mean หารด้วย std

In [42]:
# x = data, m = mean, s = standard deviation
def normalize(x, m, s): 
    return (x-m)/s

In [43]:
x_train = normalize(x_train, train_mean, train_std)

Normalize ข้อมูล Validation Set ด้วย mean, std ของ Training Set เพื่อปรับให้เป็น Scale เดียวกัน

In [44]:
x_valid = normalize(x_valid, train_mean, train_std)

ลองดูค่า mean, std หลังจาก Normalize เรียบร้อยแล้ว

In [45]:
train_mean_after, train_std_after = x_train.mean(), x_train.std()
train_mean_after, train_std_after

(tensor(-7.6999e-06), tensor(1.))

In [46]:
test_near_zero(train_mean_after)

In [47]:
test_near_zero(1-train_std_after)

In [48]:
# n = จำนวน Record, m จำนวน Feature, c = จำนวน class
n, m = x_train.shape
c = y_train.max()+1
n, m, c

(50000, 784, tensor(10))

# 4. Model

เราจะสร้าง 2 Hidden Layers Neural Networks ที่มี 50 Neuron ในแต่ละ Hidden Layer 

In [49]:
# nh = number of hidden
nh = 50

กำหนดค่าเริ่มต้นของ Weight และ Bias 

In [50]:
# Simplified Kaiming Initialize
w1 = torch.randn(m, nh)/math.sqrt(m)
b1 = torch.zeros(nh)
w2 = torch.randn(nh, 1)/math.sqrt(nh)
b2 = torch.zeros(1)

ลองดูค่า mean, std ของ Weight

In [51]:
w1.mean(), w1.std()

(tensor(-5.4121e-05), tensor(0.0360))

In [52]:
# x = data, w = weight, b = bias
def lin(x, w, b): 
    return x@w + b

In [57]:
def relu(x):
    return x.clamp_min(0.)

In [65]:
# xb = x batch, l2 = layer 2, a1 = activation 1
def model(xb):
    l0 = xb
    l1 = lin(l0, w1, b1)
    a1 = relu(l1)
    l2 = lin(a1, w2, b2)    
    return l2

yhat (y hat) คือ y ที่ได้จากการคำนวนของโมเดล ที่เราจะเอาไปเปรียบเทียบกับ y จริง ๆ จาก Training Set หรือ Validation Set

In [80]:
yhat = model(x_train)

In [81]:
yhat.shape, y_train.shape

(torch.Size([50000, 1]), torch.Size([50000]))

# 5 Loss Function

ในที่นี้เราจะใช้ Mean Squared Error เพื่อให้เข้าใจง่าย (แต่จริง ๆ ไม่เหมาะกับงาน Classification แบบนี้)

In [120]:
def mse(yhat, y): return ((yhat.squeeze(-1)-y)**2).mean()

In [121]:
y_train, y_valid = y_train.float(), y_valid.float()

In [122]:
loss = mse(yhat, y_train)
loss

tensor(26.8378)

# 6 Backpropagation and Gradients

ประกาศฟังก์ชัน Diff ของฟังก์ชันด้านบน เพื่อใช้ใน Backpropagation หา Gradient ของ Loss ที่ขึ้นกับทุก ๆ Parameter

In [158]:
# inp = input, targ = target
def mse_grad(inp, targ): 
    inp.g = (2. * (inp.squeeze()-targ).unsqueeze(-1)/inp.shape[0])
#     print(inp.shape, inp.g.shape)
    assert inp.shape == inp.g.shape

In [159]:
# inp = input, out = output
def relu_grad(inp, out): 
    inp.g = (inp>0).float() * out.g
    assert inp.shape == inp.g.shape

In [160]:
def lin_grad(inp, out, w, b): 
    inp.g = out.g @ w.t()    
    w.g = inp.squeeze(-1).t() @ out.g
    b.g = out.g.sum(0)
    assert inp.shape == inp.g.shape
    assert w.shape == w.g.shape
    assert b.shape == b.g.shape    

In [161]:
def forward_and_backward(inp, targ):
    # forward
    l1 = inp @ w1 + b1
    a1 = relu(l1)
    l2 = a1 @ w2 + b2
    loss = mse(l2, targ)
    
    # backward
    mse_grad(l2, targ)
    lin_grad(a1, l2, w2, b2)
    relu_grad(l1, a1)
    lin_grad(inp, l1, w1, b1)   

In [162]:
x_train.shape, y_train.shape

(torch.Size([50000, 784]), torch.Size([50000]))

In [163]:
forward_and_backward(x_train, y_train)

In [164]:
w1.g.mean(), w1.g.std()

(tensor(-0.0015), tensor(0.4175))

# 4/2 Refactor Model to Class

In [165]:
class Relu():
    def __call__(self, inp):
        self.inp = inp
        self.out = inp.clamp_min(0.)-0.5 # to make mean near 0
        return self.out
    def backward(self):
        self.inp.g = (self.inp>0).float()*self.out.g

In [167]:
class Lin():
    def __init__(self, w, b): self.w, self.b = w, b
        
    def __call__(self, inp):
        self.inp = inp
        self.out = inp @ self.w + self.b
    def backward(self):
        self.inp.g = self.out.g @ self.w.t()
        self.w.g = self.inp.squeeze(-1).t() @ self.out.g
        self.b.g = self.out.g.sum(0)

# Credit

* https://course.fast.ai/videos/?lesson=8
